In [82]:
import torchvision
from torchvision.io.image import read_image
from torchvision.models.detection import ssdlite320_mobilenet_v3_large, SSDLite320_MobileNet_V3_Large_Weights
from torchvision.models.detection import ssd300_vgg16, SSD300_VGG16_Weights
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
import torch
from torchvision import transforms

import cv2 as cv
from PIL import Image
import numpy as np

import glob

In [73]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Step 1: Initialize model with the best available weights
weights = SSD300_VGG16_Weights.DEFAULT
# model = fasterrcnn_resnet50_fpn_v2(weights=weights, box_score_thresh=0.9)
model = ssd300_vgg16(weights=weights)
# model.to('cuda')
model = model.eval()
print("")

Downloading: "https://download.pytorch.org/models/ssd300_vgg16_coco-b556d3b4.pth" to C:\Users\evan_/.cache\torch\hub\checkpoints\ssd300_vgg16_coco-b556d3b4.pth
100.0%


In [79]:
# img = torch.tensor(read_image("dataset/train/PXL_20201107_214721193_jpg.rf.fb52dba4cf2addec4a3d4cc2e9fa527b.jpg"), device=device)
# filepath = "dataset/train/PXL_20201107_214721193_jpg.rf.fb52dba4cf2addec4a3d4cc2e9fa527b.jpg"
# pic = Image.open(filepath)
# pic = pic.resize((320, 320))
# print(pic)
# pic.show()

# pic = pic*255
# print(pic)
# picTorch = transforms.ToTensor()(pic).unsqueeze(0)
# picTorch.resize((320, 320))
# print(type(pic))
# trans = SSDLite320_MobileNet_V3_Large_Weights.COCO_V1.transforms()
# img = trans(pic2)
# print(pic.shape)
# img = read_image("dataset/train/IMG_6818_jpg.rf.4cf2cbc673b8173c113825f52ed30a16.jpg")
img = read_image("elephant.jpg")

# Step 2: Initialize the inference transforms
preprocess = weights.transforms()
# print(preprocess(pic).shape)

# Step 3: Apply inference preprocessing transforms
batch = [preprocess(img)]

# Step 4: Use the model and visualize the prediction
prediction = model(batch)[0]
labels = [weights.meta["categories"][i] for i in prediction["labels"]]

print(prediction["boxes"][:5])
print(labels[:5])

# get score for all the predicted objects
# pred_scores = prediction['scores'].detach().cpu().numpy()
# get all the predicted bounding boxes
# pred_bboxes = prediction['boxes'].detach().cpu().numpy()
# for i in range(len(prediction["scores"])):
#     if (prediction["scores"][i] > 0.9):
#         preds
# boxes = pred_bboxes[pred_scores >= 0.90].astype(np.int32)
# boxes = torch.tensor(boxes)
# labels = prediction['labels'][:len(boxes)]
scores = prediction['scores'][:5]
print(scores)

box = draw_bounding_boxes(img, boxes=prediction["boxes"][:5],
                          labels=labels[:5],
                          colors="red",
                          width=1, font_size=30)
im = to_pil_image(box.detach())
# im.show()


# image = cv.cvtColor(np.asarray(img), cv.COLOR_BGR2RGB)
# for i, box in enumerate(boxes):
#     color = COLORS[labels[i]]
#     cv.rectangle(
#         image,
#         (int(box[0]), int(box[1])),
#         (int(box[2]), int(box[3])),
#         color, 2
#     )
#     cv.putText(image, classes[i], (int(box[0]), int(box[1]-5)),
#                 cv.FONT_HERSHEY_SIMPLEX, 0.8, color, 2, 
#                 lineType=cv.LINE_AA)


tensor([[ 335.2332,    0.0000, 1166.3341,  630.0000],
        [ 191.2635,  234.5848,  356.4376,  390.4071],
        [1013.8081,   84.4025, 1189.9844,  517.3238],
        [ 120.9551,  205.6234,  214.7919,  375.5619],
        [ 127.1045,   22.7642,  624.1195,  370.3398]],
       grad_fn=<SliceBackward0>)
['elephant', 'elephant', 'elephant', 'elephant', 'elephant']
tensor([0.9923, 0.6977, 0.0790, 0.0649, 0.0582], grad_fn=<SliceBackward0>)


In [ ]:
def save_cropped_objects (filepath, num_imgs, id):
	img = read_image(filepath)
	preprocess = weights.transforms()
	batch = [preprocess(img)]
	prediction = model(batch)[0]
	labels = [weights.meta["categories"][i] for i in prediction["labels"]]
	im = to_pil_image(box.detach())
	for i in range(num_imgs):
		print(prediction["boxes"][i].data)
		# im1 = im.crop(prediction["boxes"][i])
		# im1.save("cropped_imgs/" + str(id) + "_" + str(i) + ".jpg")

In [105]:
filepath = 'dataset/raw/train/'
for i, filename in enumerate(glob.glob(filepath + '*.jpg')):
	save_cropped_objects(filename, 5, i)

filepath = 'dataset/raw/test/'
for i, filename in enumerate(glob.glob(filepath + '*.jpg')):
	save_cropped_objects(filename, 5, i)

filepath = 'dataset/raw/valid/'
for i, filename in enumerate(glob.glob(filepath + '*.jpg')):
	save_cropped_objects(filename, 5, i)

tensor(1221.5404)
tensor(822.5319)
tensor(569.8260)
tensor(1181.4084)
tensor(743.0154)
tensor(35.4986)
tensor(283.3738)
tensor(958.6633)
tensor(525.2299)
tensor(323.8946)
tensor(1259.6219)
tensor(200.6155)
tensor(1257.3668)
tensor(1127.1907)
tensor(649.5200)
tensor(1238.7238)
tensor(1275.1406)
tensor(1204.9528)
tensor(873.5224)
tensor(751.0835)
tensor(1191.4252)
tensor(270.1425)
tensor(1051.2040)
tensor(630.8178)
tensor(0.)
tensor(408.4188)
tensor(438.9979)
tensor(1131.7271)
tensor(418.3975)
tensor(415.0886)
tensor(316.2249)
tensor(828.1139)
tensor(621.3026)
tensor(0.)
tensor(318.8970)
tensor(8.4237)
tensor(963.9153)
tensor(953.5269)
tensor(147.7668)
tensor(77.9473)
tensor(0.)
tensor(647.8837)
tensor(640.5075)
tensor(1130.3990)
tensor(228.7749)
tensor(678.6451)
tensor(270.3257)
tensor(704.4109)
tensor(69.0344)
tensor(689.8301)
tensor(497.6991)
tensor(20.5762)
tensor(8.4580)
tensor(459.1898)
tensor(459.4453)
tensor(588.6854)
tensor(518.9532)
tensor(189.0617)
tensor(639.3819)
tensor(660.